<a href="https://colab.research.google.com/github/OctavioGuaschi/IA901-2023S1/blob/main/projetos/Human_physical_position_detection/notebooks/Splitdataset_finetunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
data_dir = "gdrive/MyDrive/Projeto/data/imagens_selecionadas"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Imports

# import the needed libs

from __future__ import print_function, division
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchsummary import summary
import matplotlib.pyplot as plt
import time
import os
import copy
from google.colab import files

plt.ion()   # interactive mode

In [ ]:
class ImageFolderWithPath(datasets.ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return img, target, path

In [ ]:
image_datasets = ImageFolderWithPath(os.path.join(data_dir))
# Train (70%), Val (15%), Test (15%)
train_size = int(0.7 * len(image_datasets))
test_size = int(0.15 * len(image_datasets))
val_size = len(image_datasets) - train_size - test_size
train_dataset_1, val_dataset_1, test_dataset_1 = torch.utils.data.random_split(image_datasets, [train_size, val_size, test_size])



In [ ]:
test_size

51

In [ ]:
print(image_datasets.imgs[0])

('gdrive/MyDrive/Projeto/data/imagens_selecionadas/tmp/lying/000208151.jpg', 0)


In [ ]:
print(train_dataset_1.indices)

[57, 108, 230, 234, 34, 285, 236, 30, 240, 330, 119, 291, 93, 16, 170, 317, 269, 104, 162, 149, 3, 210, 18, 248, 17, 203, 204, 151, 50, 239, 13, 27, 322, 300, 118, 38, 8, 154, 218, 115, 33, 328, 325, 56, 202, 142, 206, 296, 297, 311, 171, 139, 1, 242, 39, 102, 46, 156, 208, 63, 143, 247, 256, 193, 321, 222, 117, 36, 220, 24, 35, 186, 106, 318, 128, 79, 226, 290, 188, 313, 0, 254, 130, 51, 284, 14, 308, 259, 303, 28, 174, 332, 100, 29, 21, 152, 90, 72, 91, 200, 180, 184, 272, 219, 144, 302, 198, 131, 237, 283, 225, 37, 255, 201, 310, 112, 78, 40, 196, 127, 312, 189, 268, 274, 265, 85, 261, 20, 42, 80, 62, 77, 257, 262, 329, 41, 2, 319, 314, 47, 179, 235, 298, 192, 53, 26, 9, 135, 339, 287, 133, 88, 92, 48, 166, 147, 238, 129, 113, 109, 264, 161, 211, 190, 279, 195, 89, 289, 243, 83, 65, 253, 227, 82, 258, 55, 270, 99, 304, 294, 305, 333, 215, 307, 336, 288, 61, 110, 160, 176, 340, 86, 267, 175, 320, 275, 81, 231, 6, 141, 277, 49, 181, 197, 150, 136, 70, 327, 67, 23, 153, 250, 326, 145, 

In [ ]:
train_dataset = train_dataset_1.indices

In [ ]:
print(val_dataset_1.indices)

[183, 164, 52, 123, 120, 191, 335, 45, 98, 54, 96, 229, 280, 260, 163, 122, 105, 252, 101, 155, 169, 32, 233, 306, 173, 251, 323, 281, 334, 221, 273, 68, 276, 157, 4, 159, 178, 263, 121, 84, 12, 66, 266, 337, 43, 94, 167, 338, 199, 95, 58, 223]


In [ ]:
val_dataset = val_dataset_1.indices

In [ ]:
print(test_dataset_1.indices)

[245, 97, 146, 228, 148, 207, 309, 74, 182, 217, 185, 244, 315, 126, 116, 5, 22, 64, 299, 205, 134, 241, 59, 293, 60, 282, 138, 187, 87, 194, 114, 286, 331, 25, 140, 31, 212, 278, 292, 11, 216, 15, 76, 165, 213, 271, 107, 73, 71, 224, 316]


In [ ]:
test_dataset = test_dataset_1.indices

In [ ]:
for i in range(0, train_size):
  print( str(i) + " " + image_datasets.imgs[train_dataset[i]][0].split("gdrive/MyDrive/Projeto/data/imagens_selecionadas/tmp")[1])
  img = Image.open(image_datasets.imgs[train_dataset[i]][0])
  img.save(('gdrive/MyDrive/Projeto/data/processed/DatasetFineTunning/train' + image_datasets.imgs[train_dataset[i]][0].split("gdrive/MyDrive/Projeto/data/imagens_selecionadas/tmp")[1]))

0 /lying/025346914.jpg
1 /lying/098543738.jpg
2 /standing/000024100.jpg
3 /standing/000041029.jpg
4 /lying/015124683.jpg
5 /standing/002576960.jpg
6 /standing/000111209.jpg
7 /lying/012890259.jpg
8 /standing/000147382.jpg
9 /standing/005622429.jpg
10 /sitting/0004373681.jpg
11 /standing/003095579.jpg
12 /lying/089025805.jpg
13 /lying/006964115.jpg
14 /sitting/004433225.jpg
15 /standing/004930899.jpg
16 /standing/001272728.jpg
17 /lying/096738599.jpg
18 /sitting/003490586.jpg
19 /sitting/002619671.jpg
20 /lying/001344404.jpg
21 /sitting/007269201.jpg
22 /lying/007424686.jpg
23 /standing/000383029.jpg
24 /lying/007189942.jpg
25 /sitting/006380474.jpg
26 /sitting/006616872.jpg
27 /sitting/002753672.jpg
28 /lying/020679637.jpg
29 /standing/000144722.jpg
30 /lying/006272159.jpg
31 /lying/012680180.jpg
32 /standing/005274395.jpg
33 /standing/003820278.jpg
34 /sitting/000437064.jpg
35 /lying/016949275.jpg
36 /lying/002944251.jpg
37 /sitting/002919084.jpg
38 /sitting/008238125.jpg
39 /sitting/

In [ ]:
train_dataset[i]

10

In [ ]:
for i in range(0, test_size):
  print( str(i) + " " + image_datasets.imgs[test_dataset[i]][0].split("gdrive/MyDrive/Projeto/data/imagens_selecionadas/tmp")[1])
  img = Image.open(image_datasets.imgs[test_dataset[i]][0])
  img.save(('gdrive/MyDrive/Projeto/data/processed/DatasetFineTunning/test' + image_datasets.imgs[test_dataset[i]][0].split("gdrive/MyDrive/Projeto/data/imagens_selecionadas/tmp")[1]))

0 /standing/000302087.jpg
1 /lying/093202811.jpg
2 /sitting/002394627.jpg
3 /standing/000013469.jpg
4 /sitting/002608329.jpg
5 /sitting/006835320.jpg
6 /standing/004685191.jpg
7 /lying/069923899.jpg
8 /sitting/004916083.jpg
9 /sitting/008238105.jpg
10 /sitting/005006453.jpg
11 /standing/000298013.jpg
12 /standing/004904989.jpg
13 /sitting/000956171.jpg
14 /sitting/000418744.jpg
15 /lying/002058449.jpg
16 /lying/010193191.jpg
17 /lying/030980688.jpg
18 /standing/003778451.jpg
19 /sitting/006640535.jpg
20 /sitting/001541496.jpg
21 /standing/000187274.jpg
22 /lying/026517074.jpg
23 /standing/003282550.jpg
24 /lying/026720728.jpg
25 /standing/002335210.jpg
26 /sitting/001873029.jpg
27 /sitting/005160254.jpg
28 /lying/085573726.jpg
29 /sitting/005476378.jpg
30 /sitting/000099186.jpg
31 /standing/002581191.jpg
32 /standing/005630328.jpg
33 /lying/012032666.jpg
34 /sitting/002098498.jpg
35 /lying/013008624.jpg
36 /sitting/007560159.jpg
37 /standing/002098300.jpg
38 /standing/003152219.jpg
39 

In [ ]:
for i in range(0, val_size):
  print( str(i) + " " + image_datasets.imgs[val_dataset[i]][0].split("gdrive/MyDrive/Projeto/data/imagens_selecionadas/tmp")[1])
  img = Image.open(image_datasets.imgs[val_dataset[i]][0])
  img.save(('gdrive/MyDrive/Projeto/data/processed/DatasetFineTunning/val' + image_datasets.imgs[val_dataset[i]][0].split("gdrive/MyDrive/Projeto/data/imagens_selecionadas/tmp")[1]))

0 /sitting/004922091.jpg
1 /sitting/003826414.jpg
2 /lying/021143126.jpg
3 /sitting/000754109.jpg
4 /sitting/0004373682.jpg
5 /sitting/005334800.jpg
6 /standing/006577670.jpg
7 /lying/018741194.jpg
8 /lying/093539449.jpg
9 /lying/023568253.jpg
10 /lying/091356256.jpg
11 /standing/000024087.jpg
12 /standing/002194147.jpg
13 /standing/000982755.jpg
14 /sitting/003598983.jpg
15 /sitting/000648808.jpg
16 /lying/096831921.jpg
17 /standing/000550580.jpg
18 /lying/095652775.jpg
19 /sitting/003039741.jpg
20 /sitting/004429543.jpg
21 /lying/013107986.jpg
22 /standing/000040154.jpg
23 /standing/004027816.jpg
24 /sitting/004449443.jpg
25 /standing/000506158.jpg
26 /standing/005433203.jpg
27 /standing/002225013.jpg
28 /standing/006243133.jpg
29 /sitting/012839429.jpg
30 /standing/001522247.jpg
31 /lying/033362840.jpg
32 /standing/001845642.jpg
33 /sitting/003117491.jpg
34 /lying/001808109.jpg
35 /sitting/003355918.jpg
36 /sitting/004778136.jpg
37 /standing/001073898.jpg
38 /sitting/000563328.jpg
3

In [ ]:
print(train_dataset.indices[:])

AttributeError: ignored

In [ ]:
print(test_dataset.indices[:])

AttributeError: ignored

In [ ]:
print(val_dataset.indices[:])

AttributeError: ignored

17 de maio de 2023 as 17h25